## Загрузка данных

In [1]:
import gensim
from gensim.models import Word2Vec
import re
import pymorphy2
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
# from my_parsers import download, parse
from tqdm import tqdm
from collections import Counter
# from brown_clustering import BigramCorpus, BrownClustering
from util_functions import *
pd.options.display.max_rows = 200
tqdm.pandas()

C:\Users\iltya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\iltya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\iltya\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\iltya\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_TRAIN_RES_1 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')

In [3]:
TRAIN_VECTORIZERS = True    # обучать заново векторизаторы

## Начальная предобработка данных

In [4]:
# удаляем модифицированный столбец - он совпадает с точностью до регистра
df_TRAIN_RES_1.drop("achievements_modified", axis=1, inplace=True)

for text_feature in ["achievements", "company_name", "demands"]:
    df_TRAIN_RES_1[text_feature].fillna("", inplace=True)

df_TRAIN_RES_1["achievements_len"] = [len(i) for i in df_TRAIN_RES_1["achievements"]]
df_TRAIN_RES_1.drop_duplicates(inplace=True)

In [5]:
DROP_PART_DUPLICATES = False

if DROP_PART_DUPLICATES:
    df_TRAIN_RES_1["comp_name_demands"] = df_TRAIN_RES_1["demands"] + df_TRAIN_RES_1["company_name"]
    df_TRAIN_RES_1 = df_TRAIN_RES_1.drop_duplicates(subset="comp_name_demands")
    df_TRAIN_RES_1.drop("comp_name_demands", axis=1, inplace=True)

In [6]:
vc = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False)
print("proportion of samples choised clases: ", vc.iloc[:200].values.sum() / vc.values.sum())

proportion of samples choised clases:  0.9580346140836676


In [7]:
used_classes = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False).iloc[:200].index.tolist()
df_TRAIN_RES_1 = df_TRAIN_RES_1[df_TRAIN_RES_1["job_title"].isin(used_classes)]
# df_TRAIN_RES_1 = df_TRAIN_RES_1.iloc[::5]
df_TRAIN_RES_1["len_group"] = df_TRAIN_RES_1.groupby("id_cv").agg({"company_name": "count"})

In [8]:
df_TRAIN_RES_1.head()

,achievements,company_name,demands,id_cv,job_title,achievements_len,len_group
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN
1,,Администрация города Сердобска,Работа с реестром муниципальной собственности,52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN
2,"Работы не закончены, т.к. нет настоящего желан...",январь 2016-настоящее времяООО «АРТЕЛЬ СТАРАТЕ...,Курирую работы по рудному направлению: соавтор...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,185,NaN
3,,"ОАО «Васильевский рудник», п.Мотыгино, Красноя...",ГРП. Документация керна разведочных скважин,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN
4,,"ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",Поисково-разведочные работы на месторождении р...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN


## w2v for company names

In [9]:
if TRAIN_VECTORIZERS:
    company_names = [i["company_name"] + " " + i["job_title"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)

    # Обучение модели Word2Vec
    model_comp_name = Word2Vec(sentences, vector_size=64, window=4, min_count=1, sg=1)
    # Сохранение обученной модели
    model_comp_name.save("models/company_word2vec_russian.model")
else:
    company_names = [i["company_name"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)

    model_comp_name = Word2Vec.load("models/company_word2vec_russian.model")

# model_comp_name.wv.most_similar('компьютер')

100%|██████████| 1594617/1594617 [00:08<00:00, 192998.52it/s]


In [12]:
comp_names_without_job = preprocess_names(df_TRAIN_RES_1["company_name"])
emb_comp_name_df = get_emb_by_modele(model_comp_name, comp_names_without_job,
                                     column_prefix="comp_name_emb", vector_size=64)
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_comp_name_df.reset_index(drop=True)], axis=1)

100%|██████████| 1594617/1594617 [00:27<00:00, 57153.47it/s]


## w2v for demands

In [14]:

if TRAIN_VECTORIZERS:
    company_names = [i["demands"] + " " + i["job_title"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)

    # Обучение модели Word2Vec
    model_demand = Word2Vec(sentences, vector_size=100, window=8, min_count=1, sg=1)
    # Сохранение обученной модели
    model_demand.save("models/demand_word2vec_russian.model")
else:
    company_names = [i["demands"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)
    model_demand = Word2Vec.load("models/demand_word2vec_russian.model")

model_demand.wv.most_similar('бухгалтер')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\iltya\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\iltya\AppData\Local\Temp/ipykernel_17340/2148372934.py", line 11, in <module>
    company_names = [i["demands"] for _, i in df_TRAIN_RES_1.iterrows()]
  File "C:\Users\iltya\AppData\Local\Temp/ipykernel_17340/2148372934.py", line 11, in <listcomp>
    company_names = [i["demands"] for _, i in df_TRAIN_RES_1.iterrows()]
  File "C:\Users\iltya\anaconda3\lib\site-packages\pandas\core\frame.py", line 1262, in iterrows
    for k, v in zip(self.index, self.values):
  File "C:\Users\iltya\anaconda3\lib\site-packages\pandas\core\frame.py", line 10664, in values
    return self._mgr.as_array(transpose=True)
  File "C:\Users\iltya\anaconda3\lib\site-packages\pandas\core\internals\managers.py", line 1466, in as_array
    arr = self._interleave(dtype=dtype, na_value=na_value)
  File "C:

TypeError: object of type 'NoneType' has no len()

In [ ]:
demand_without_job = preprocess_names(df_TRAIN_RES_1["demands"])
emb_demand_df = get_emb_by_modele(model_demand, demand_without_job, column_prefix="demand_emb")
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_demand_df.reset_index(drop=True)], axis=1)

In [ ]:
df_TRAIN_RES_1.head()

## add tf_idf company name

In [ ]:
sentences = preprocess_names(df_TRAIN_RES_1["company_name"])

column_prefix = "comp_name_tfidf"

if TRAIN_VECTORIZERS:
    comp_name_vectorizer, comp_name_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                             column_prefix=column_prefix)
else:
    comp_name_vectorizer = pickle.load(open("models/comp_name_vectorizer.pkl", "rb"))

    comp_name_tfidf = comp_name_vectorizer.transform([" ".join(sent) for sent in sentences])
    comp_name_tfidf = pd.DataFrame(comp_name_tfidf.toarray(),
                                   columns=[f"{column_prefix}_{word}" for word in comp_name_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            comp_name_tfidf.reset_index(drop=True)], axis=1)

In [ ]:
sentences = preprocess_names(df_TRAIN_RES_1["demands"])

column_prefix = "demnds_tfidf"

if TRAIN_VECTORIZERS:
    demands_vectorizer, demands_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)
else:
    demands_vectorizer = pickle.load(open("models/demands_vectorizer.pkl", "rb"))

    demands_tfidf = demands_vectorizer.transform([" ".join(sent) for sent in sentences])
    demands_tfidf = pd.DataFrame(demands_tfidf.toarray(),
                                 columns=[f"{column_prefix}_{word}" for word in demands_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            demands_tfidf.reset_index(drop=True)], axis=1)

In [ ]:
sentences = preprocess_names(df_TRAIN_RES_1["achievements"])

column_prefix = "achiv_tfidf"

if TRAIN_VECTORIZERS:
    achiv_vectorizer, achiv_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)
else:
    achiv_vectorizer = pickle.load(open("models/achiv_vectorizer.pkl", "rb"))
    achiv_tfidf = achiv_vectorizer.transform([" ".join(sent) for sent in sentences])
    achiv_tfidf = pd.DataFrame(achiv_tfidf.toarray(),
                               columns=[f"{column_prefix}_{word}" for word in demands_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            achiv_tfidf.reset_index(drop=True)], axis=1)

In [ ]:
df_TRAIN_RES_1.head()

In [ ]:
# with open("comp_name_vectorizer.pkl", "wb") as f:
#     pickle.dump(comp_name_vectorizer, f)
#
# with open("demands_vectorizer.pkl", "wb") as f:
#     pickle.dump(demands_vectorizer, f)
#
# with open("achiv_vectorizer.pkl", "wb") as f:
#     pickle.dump(achiv_vectorizer, f)


## train model

In [ ]:
df_TRAIN_RES_1 = df_TRAIN_RES_1.iloc[::5]

In [ ]:
columns_to_drop = ["achievements", "company_name", "demands", "id_cv",]

In [ ]:
X, y = df_TRAIN_RES_1.drop(columns_to_drop + ["job_title"], axis=1), df_TRAIN_RES_1["job_title"]

In [ ]:
from sklearn.model_selection import train_test_split
N = 20_000
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:N], y.iloc[:N], stratify=y.iloc[:N], test_size=0.2, random_state=42)

In [ ]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

In [ ]:
clf = CatBoostClassifier(task_type="GPU", eval_metric="Accuracy")
clf.fit(train_pool, eval_set=test_pool, early_stopping_rounds=5)

In [39]:
with open("clf.pkl", "wb") as f:
    pickle.dump(clf, f)

In [47]:
from collections import Counter
Counter([i[0] for i in clf.predict(X_test).tolist()])

Counter({'менеджер': 342,
         'электрогазосварщик': 14,
         'оператор': 223,
         'водитель': 130,
         'юрисконсульт': 91,
         'специалист': 686,
         'кладовщик': 38,
         'директор': 154,
         'мастер': 108,
         'администратор': 87,
         'кассир': 40,
         'слесарь': 93,
         'психолог': 14,
         'инженер': 444,
         'бухгалтер': 287,
         'руководитель': 42,
         'продавец': 94,
         'машинист': 67,
         'электромонтер': 73,
         'заместитель': 99,
         'экономист': 72,
         'фельдшер': 11,
         'консультант': 15,
         'инспектор': 54,
         'секретарь': 77,
         'архивариус': 1,
         'учитель': 77,
         'сборщик': 4,
         'механик': 34,
         'укладчик': 2,
         'медсестра': 7,
         'официант': 8,
         'монтажник': 37,
         'педагог': 14,
         'преподаватель': 76,
         'охранник': 23,
         'юрист': 22,
         'воспитатель': 42,
       